In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image 

import datetime as dt
import scipy as sp
import statsmodels.api as sm
import pandas as pd

import sklearn.cluster


In [ ]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    !git clone https://github.com/rkbono/GLY4451.git
    !pip install cartopy
    fpath = './GLY4451/'
else:
    print('Not running on CoLab')
    fpath = './'

# Agenda
1. Intro to grids and np.meshgrid
1. Contour plots
1. Linear regresion (by hand)
1. Linear regression (with scipy)
1. Quick bootstrap
1. Quick monte carlo
1. Cluster analysis
1. Searching for global minimum

# Grids and np.meshgrid

Numpy meshgrid is a tool for numeric data manipulation in Python.

We use Numpy meshgrid to create a rectangular grid of x and y values.

More specifically, meshgrid creates coordinate values that enable us to construct a rectangular grid of values.

It does this in a somewhat roundabout way.

As inputs to the function, we provide 1-dimensional arrays with numeric values. These numeric values will be the coordinates of the new “meshgrid.”

The output of np.meshgrid is a set of Numpy arrays that contain the coordinates of this new grid space.

In [ ]:
Image(fpath+'Figures/meshgrid2d.png')

# Contour plots


A contour line or isoline of a function of two variables is a curve along which the function has a constant value.

It is a cross-section of the three-dimensional graph of the function f(x, y) parallel to the x, y plane.

A contour plot is appropriate if you want to see how value Z changes as a function of two inputs X and Y, such that Z = f(X,Y). A contour line or isoline of a function of two variables is a curve along which the function has a constant value.

The independent variables x and y are usually restricted to a regular grid called meshgrid. The numpy.meshgrid creates a rectangular grid out of an array of x values and an array of y values.

Matplotlib API contains contour() and contourf() functions that draw contour lines and filled contours, respectively. Both functions need three parameters x,y and z.

In [ ]:
xlist = np.linspace(-3.0, 3.0, 100)
ylist = np.linspace(-3.0, 3.0, 100)
X, Y = np.meshgrid(xlist, ylist)
Z = np.sqrt(X**2 + Y**2)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(X, Y, Z)
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')
ax.set_aspect('equal')

In [ ]:
xlist = np.linspace(-3.0, 3.0, 100)
ylist = np.linspace(-3.0, 3.0, 100)
X, Y = np.meshgrid(xlist, ylist)
Z = np.sqrt(X**2 + Y**2)
fig,ax=plt.subplots(1,1)
cp = ax.contour(X, Y, Z)
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')
ax.set_aspect('equal')

In [ ]:
xlist = np.linspace(-3.0, 3.0, 100)
ylist = np.linspace(-3.0, 3.0, 100)
X, Y = np.meshgrid(xlist, ylist)
Z = np.sqrt(X**2 + Y**2)
fig,ax=plt.subplots(1,1)
cp = ax.contourf(X, Y, Z,cmap=plt.cm.turbo,levels=100)
ax.contour(X,Y,Z,colors='k')
fig.colorbar(cp) # Add a colorbar to a plot
ax.set_title('Filled Contours Plot')
ax.set_xlabel('x (cm)')
ax.set_ylabel('y (cm)')
ax.set_aspect('equal')

## Challenge:

Plot temperature with respect to age and depth for the crust near a spreading ridge

In [ ]:
# Function to get temp given depth, age, initial temp and diffusitivity
def halfspace_temp(z,t,ta,ka):
    
    return ta*sp.special.erf(z/(2*np.sqrt(ka*t)))

# age from myr to seconds
fMyr = lambda t: dt.timedelta(weeks=52).total_seconds()*1e6*t

In [ ]:
# constants
ka = 1e-6 # diffusivity
t0 = 1300 # initial temp

# initial vector of ages
tmyr = np.arange(0,100.1,1)

# initial vector of depths
dz = np.arange(0,150e3,1e3)

# colormap
cmap = plt.cm.magma

# levels for contours
lev = np.arange(0,1300+1e-5,100)

In [ ]:
Image(fpath+'Figures/heat_contourf.png')

# Linear regression
## By "hand"

Let's put into practice what we talked about on Monday. Using numpy, let's implement our own version of a linear regression.

Recall:

$\beta = (X^{T}X)^{-1}X^{T}y$

So for a given set of observations **y** and independent regressor **x** we'd like to fit model parameters $\beta$. 

Let's work with just a simple curve (degree 2 polynomial) with the form:

$y = \beta_{0} + \beta_{1}x + \beta_{2}x^2$

This makes **X** our design matrix easy to setup. For each observation $y_i$, **X** will have a row containing [1, $x_i$] corresponding to the two model parameters in $\beta$. 

We can use **np.linalg** package to perform linear algebra operations as we solve for $\beta$

In [ ]:
# Define some model parameters to find
beta_true = [1, 0.5,0.25]

# and a quick line function
fxy = lambda x: beta_true[0]+beta_true[1]*x+beta_true[2]*x**2
xv = np.linspace(0,20,100) # and simple line vector

# Let's generate some sample data with noise
xobs = np.random.uniform(0,20,size=30)
yobs = fxy(xobs)

# and take a look at the data
plt.plot(xobs,yobs,'ok',label='obs')
plt.plot(xv,fxy(xv),'-',label='true')

In [ ]:
# Now lets set up our design matrix
X = np.vstack([np.ones_like(xobs),xobs,xobs**2]).T # added transpose

# and regress!
norm_matrix = np.linalg.inv(np.dot(X.T,X))
beta_guess = np.linalg.multi_dot([norm_matrix,X.T,yobs.T])
print(beta_guess)

beta_regress_np = np.linalg.lstsq(X,yobs,rcond=None)
print(beta_regress_np[0])

## Add some error

In [ ]:
np.random.seed(1397)

# Let's generate some sample data with noise
xobs = np.random.uniform(0,20,size=30)
yobs = fxy(xobs)+np.random.normal(scale=5,size=xobs.shape) # random term adds Gaussian noise

# and take a look at the data
plt.plot(xobs,yobs,'ok',label='obs')
plt.plot(xv,fxy(xv),'-',label='true')

In [ ]:
# Now lets set up our design matrix
X = np.vstack([np.ones_like(xobs),xobs,xobs**2]).T # added transpose

# and regress!
norm_matrix = np.linalg.inv(np.dot(X.T,X))
beta_guess = np.linalg.multi_dot([norm_matrix,X.T,yobs.T])
print(beta_guess)

beta_regress_np = np.linalg.lstsq(X,yobs,rcond=None)
print(beta_regress_np[0])

## Calculate $R^2$

In [ ]:
ssr = np.sum([(yi-(beta_guess[0]+beta_guess[1]*xi+beta_guess[2]*xi**2))**2 for yi,xi in zip(yobs,xobs)])
print(ssr,beta_regress_np[1][0])

sst = np.sum((yobs-np.mean(yobs))**2)

r2 = 1-(ssr/sst)

print('R^2 = %.3f'%r2)

## Plot fit

In [ ]:
plt.plot(xobs,yobs,'ok',label='obs')
plt.plot(xv,fxy(xv),'-',label='true')
plt.plot(xv,[(beta_guess[0]+beta_guess[1]*xi+beta_guess[2]*xi**2) for xi in xv],'-',label='fit, $R^2$: '+'%.3f'%r2)
plt.legend()

# Bootstrapping!

In [ ]:
# Now lets set up our design matrix
X = np.vstack([np.ones_like(xobs),xobs,xobs**2]).T # added transpose

# lets functionalize our regression
freg = lambda X,y: np.linalg.multi_dot([np.linalg.inv(np.dot(X.T,X)),X.T,y.T])

### Bootstrapping consists of the following steps:
1. Define a desired statistic(s) and a collector to hold the results
1. Set up a for loop to interate the desired number of realizations (ndraw >= 2500)
1. In each loop, draw with replacement from the observations to generate a synthetic dataset
1. With the synthetic dataset, perform statistics and collect result
1. After all fits are done, examine the resulting stats
1. ???
1. Profit!

In [ ]:
ndraw = 10000

res = []

for ii in range(ndraw):
    ridx = np.random.randint(0,X.shape[0],size=X.shape[0])
    Xt = X[ridx,:]
    yt = yobs[ridx]
    res.append(freg(Xt,yt))
res = np.array(res)

In [ ]:
fig = plt.figure(figsize=(14,4))
ax = fig.subplots(1,3)

for ii in range(3):
    ax[ii].hist(res[:,ii],bins=100)
    ax[ii].axvline(beta_true[ii],color='red',linewidth=2,linestyle='--',label='True')
    [ax[ii].axvline(pp,color='k') for pp in np.percentile(res[:,ii],[2.5,50,97.5])]
    
    ax[ii].set_title(r'$\beta_{%d}$'%ii)

In [ ]:
fig = plt.figure(figsize=(8,6))
ax = fig.subplots(1,1)

ax.plot(xobs,yobs,'ob',mec='w',zorder=100)
ax.plot(xv,fxy(xv),'-r',label='True',zorder=10)

for rr in res[::10]:
    ax.plot(xv,(rr[0]+rr[1]*xv+rr[2]*xv**2),'-',color='grey',linewidth=0.5,alpha=0.1,zorder=0)
    
ax.legend()

# Quick Monte Carlo

## Integration

In [ ]:
xrng = [0,1]
yrng = [0,1]

fcirc = lambda x,y: True if (x**2 + y**2 <= 1) else False

rnd = np.random.random(size=[int(1e3),2])
counts = np.array([[x[0],x[1],fcirc(x[0],x[1])] for x in rnd])

fig = plt.figure()
ax = fig.subplots(1,1)

ax.plot(counts[counts[:,2]==1,0],counts[counts[:,2]==1,1],'.b')
ax.plot(counts[counts[:,2]==0,0],counts[counts[:,2]==0,1],'.r')

ax.plot(np.cos(np.linspace(0,np.pi/2,100)),np.sin(np.linspace(0,np.pi/2,100)),'-k')

ax.set_aspect('equal')
ax.set_xlim(xrng)
ax.set_ylim(yrng)

print(4*np.sum(counts[:,2])/counts.shape[0],np.pi)

## Challenge - convergence

Goal: Estimate how many Mone Carlo realizations are needed to get an accurate estimate of pi. Consider how close is close enough? What's an efficient way to explore the question? How variable is the estimate?

## Challenge - Monte Carlo Simulation

Goal: estimate the geotherm temperature at 20 km depth given surface heat flow and internal heat generation. But both observations are means w/ uncertainty based on several measurements that have been averaged together, so your temperature estimate should include that uncertainty. 

In [ ]:
# function for getting temp from equilibrium geotherm and q0 at the surface
def geotherm_surf(z,q,a,k):
    """
    returns temp (degC) given depth z (m), surf heat flow q, internal heat a and conductivity k
    """
    return -(a/(2*k))*z**2 + q/k*z

# constants
k = 2.5 # conductivity
q0 = {'mean':0.058,'std':0.01}
A = {'mean':1.25e-6,'std':0.250e-6}

In [ ]:
# look at geotherm assuming mean values
zz = np.linspace(0,30e3,100)

tz = geotherm_surf(zz,q0['mean'],A['mean'],k)

plt.plot(tz,zz/1e3,'-r')
plt.ylim([30,0])
plt.xlabel('Temp (°C)')
plt.ylabel('Depth (km)')

In [ ]:
Image(fpath+'Figures/q0_a_montecarlo.png')

In [ ]:
Image(fpath+'Figures/geotherm_mc.png')

In [ ]:
Image(fpath+'Figures/temp_at_20km.png')

# Clustering

We'll be reproducing an example of using k-means clustering to group a set of structural data on either side of a fold axis.

### The Orocopio Mountains Dataset
The dataset poles_data contains a dataset of poles to bedding planes from the Orocopio mountains. We learned about poles to planes in Lecture 22. If a rock is composed of sediments that are layed down flat on top of one another, then we would expect the pole to the plane to be vertical (because the plane itself is horizontal). If instead the plane is tilted, we might expect the pole to the plane to be in some other direction. Let's peek at a data set of poles from bedding planes measured in the  Orocopio Mountains.

In [ ]:
poles_data=pd.read_csv(fpath+'Datasets/Orocopio_Poles_Data.csv')
poles_data.head()

We can use a modified equal area plot to plot the 'Pole_Az' and 'Pole Plunge' columns on an equal area projection to see what the bedding planes look like.

In [ ]:
# Here's the equal angle function
EqualArea = lambda Pl: np.sqrt(2.)*90.*np.sin(np.radians(90.-Pl)/(2.))

def plot_equal_area(Azs,Pls,colors='black',cmap='RdBu',alpha=1):
    """
    Plots an Equal Angle plot for data d, colors are a string or list of colors
    to be passed to the points in data
    Note that this is different to the code in lecture 20 because it uses a
    scatter plot.
    
    Parameters:
    ____________
        Azs : np.array
            Azimuths of poles to planes
        Pls : np.array
            Plunges of poles to planes
            
    Returns:
    _________
        None
             
    """
    fig = plt.subplot(111, polar=True) 
    # set the coordinates (like for rose diagrams)
    fig.set_theta_direction(-1) # Reverse direction of degrees (CW)
    fig.set_theta_zero_location("N") # Specify 0-degrees as North
    # for this we want the full 90 degrees, so set the scale
    plt.polar([0],[90]) ## to scale grid
    # plot the first direction as a red dot
    fig.scatter(np.radians(Azs),(EqualArea(Pls)),c=colors,cmap=cmap,alpha=alpha)
    # plot the second direction as a blue square

    # make a list of contours to plot
    # notice use of list comprehension
    # label the azimuths at 20 degree intervals
    AzContours=range(0,360,20)
    AzLabels=[str(p) for p in AzContours]
    plt.thetagrids(AzContours,AzLabels)
    # and now the plunges
    PlContours=[EqualArea(a) for a in range(10,90,20)] ##don't include center or edge
    # make a list of labels
    PlLabels=[str(a) for a in range(10,90,20)]
    # draw on the plunge contours and label them
    plt.rgrids(PlContours,PlLabels)
    # label the plot
    plt.title('Equal Area Net');

In [ ]:
plot_equal_area(poles_data.Pole_Az,poles_data.Pole_Plunge,colors='cyan')

This is interesting! It seems that there are two 'clusters' with of bedding planes in different directions in this dataset, one to the north-east and one to the south-west. We want a way of separating these two clusters, but first let's think about what causes this. Is there some spatial relationship between where the different directions are found?

#### Quiver Plots and **plt.imshow( )**

To illustrate this we can use a type of plot known as a 'quiver plot'.  We will learn more about quiver plots in later lectures.  But for now, it draws an arrow with the direction of the plane on a plot. To do this, we need to convert the data from azimuth and plunge to x, y and z. We have the handy function **dir2cart( )** which we already know about.  

In [ ]:
def dir2cart(Az,Pl):
    """
    converts polar directions to cartesian coordinates
    Inputs: 
        Dir[Azimuth,Plunge]:  directions in degreess
    Output:
        [X,Y,Z]: cartesian coordinates
    """
    Az=np.radians(Az)
    Pl=np.radians(Pl)
    return [np.cos(Az)*np.cos(Pl),np.sin(Az)*np.cos(Pl),np.sin(Pl)]

In [ ]:
# get cartesian form of pole vectors
u,v,w=dir2cart(poles_data.Pole_Az.values,poles_data.Pole_Plunge.values)

In our coordinate system, $w$ is straight up, so planes with a steeper direction will have a smaller $u$ and $v$ components and a larger $w$ component, and so the arrows on the quiver plot will appear shorter in length.  

We will plot the quiver plot on top of a satellite image of the area, using the **plt.imread( )** and **plt.imshow( )** functions in **matplotlib**. These take an image and convert it into a coordinate system we can plot data onto.

In [ ]:
img = plt.imread(fpath+'Figures/GoogleEarthImage.png') #Reads in our image as a numpy array
extent = [-115.7115, -115.6795, 33.5442, 33.5651] #Sets the corners of the image in lat/lon for plotting
plt.figure(figsize=(9,13)) #Creates a new figure object to put the image on
plt.imshow(img, origin='upper', extent=extent) #Plots the satellite image.;

#Now let's plot the quivers onto the image 
#plt.quiver takes 4 arguments, x and y (locations of arrows), 
# and u and v (lengths of arrows in u and v directions).  
# We can also set the color so we can see the vectors better

plt.quiver(poles_data.Lon,poles_data.Lat,u,v,color='cyan');

This plot tells us an interesting story. Along the center of the satellite image runs a linear feature. To the north of this feature, we see that the arrows are pointing to the north-east. To the south-west of this image, the arrows are pointing south-west. What could be the cause of this pattern?

One probable cause would be a fold or anticline. For an illustration, see the image below. In an anticline, the horizontal layers are tilted away from the axis of the fold, so that the poles to the plane (arrows) are pointing away from the fold axis (dotted line). 

In [ ]:
# Image('Figures/Fold_Diagram.png',width=300)

#### Clustering our data
Instead of "eyeballing" as we did at first, what if we wanted to automatically sort the two different directions into two different groups? How would we most easily do that? We don't really want to have to _train_ this dataset as we don't really care which group is which in this case, we just want some way of splitting the data into sensible groups. As such we might want to use some kind of _unsupervised_ machine learning process.

The **scikit-learn** package has a module called **sklearn.cluster** that allows us to solve this problem. There are many algorithms for different 'shapes' of clusters. Let's try converting our data into a format **scikit-learn** understands, then use the **Kmeans** clustering algorithm on them.

Remember from Lecture 16, that **scikit-learn** requires our data to be in a format in which  each datapoint has a set of _features_ which are a bit like coordinates.

In [ ]:
input_data=np.array([poles_data.Pole_Az,poles_data.Pole_Plunge]).T
print(input_data[0:5])

Now let's do the clustering:

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=2) #This tells us that we are using a clustering algorithm with 2 clusters

fit=kmeans.fit(input_data) #Fits the kmeans algorithm to our input data
clusternumbers=kmeans.predict(input_data) #Gives the cluster numbers for each of our clusters
#Plots the equal area with colors for clusters
plot_equal_area(poles_data.Pole_Az,poles_data.Pole_Plunge,colors=clusternumbers)

Hmm, it seems like this didn't work exactly as expected. Notice how there seems to be a change in cluster across the 0 degree Azimuth line? Let's plot Azimuth against plunge on an x,y plot to see why this didn't seem to work very well. 

In [ ]:
plt.scatter(poles_data.Pole_Az,poles_data.Pole_Plunge,c=clusternumbers,cmap='RdBu');
plt.xlabel('Azimuth')
plt.ylabel('Plunge');

The **Kmeans** algorithm treats data as if they were cartesian. But in geology, we often use directions that go from 0 to 360 which doesn't behave the same way as other cartesian data sets.  For example,an azimuth of 340 is closer to 200 than to 0 under this scheme. A simple solution to this would be to convert our azimuths and plunges to cartesian coordinates (as we did for the quiver plot) before clustering. Let's try again:

In [ ]:
kmeans = sklearn.cluster.KMeans(n_clusters=2) #This tells us that we are using a clustering algorithm 
# with 2 clusters
input_data=np.array([u,v,w]).transpose() # make and array with u,v,w as the first, second and third rows
fit=kmeans.fit(input_data) #Fits the kmeans algorithm to our input data
clusternumbers=kmeans.predict(input_data) #Gives the cluster numbers for each of our clusters
#Plots the equal area with colors for clusters
plot_equal_area(poles_data.Pole_Az,poles_data.Pole_Plunge,colors=clusternumbers)

Much better! Let's see how it looks on the satellite image!

In [ ]:
extent = [-115.7115, -115.6795, 33.5442, 33.5651]
img = plt.imread(fpath+'Figures/GoogleEarthImage.png')
plt.figure(figsize=(9,13))
plt.imshow(img, origin='upper', extent=extent)
plt.quiver(poles_data.Lon,poles_data.Lat,u,v,clusternumbers,cmap='RdBu'); #5th argument controls arrow color

Really cool.  It seems that there's something a bit more complicated going on here than just a single fold axis going down the middle, but we can see the broad trend and could probably even draw the axis in a lot of places now.
One final thing to note with **Kmeans**; you are not required  to choose the number of clusters.  However letting it work on its own generally doesn't work too well as it will try to find clusters with very similar sizes. If we try it with this example, we get a lot of clusters which don't really tell us much.

In [ ]:
kmeans = sklearn.cluster.KMeans() #unspecified number of clusters
input_data=np.array([u,v,w]).transpose()
fit=kmeans.fit(input_data) #Fits the kmeans algorithm to our input data
clusternumbers=kmeans.predict(input_data) #Gives the cluster numbers for each of our clusters
#Plots the equal area with colors for clusters
plot_equal_area(poles_data.Pole_Az,poles_data.Pole_Plunge,colors=clusternumbers,cmap='Accent')

In [ ]:
extent = [-115.7115, -115.6795, 33.5442, 33.5651]
img = plt.imread(fpath+'Figures/GoogleEarthImage.png')
plt.figure(figsize=(9,13))
plt.imshow(img, origin='upper', extent=extent)
plt.quiver(poles_data.Lon,poles_data.Lat,u,v,clusternumbers,cmap='Accent');

# Searching for the global minimum

Here we will try out some different approaches to find the global minimum for challenging equations

### Rosenbrock function

In [ ]:
frb = lambda x: (x[0]-1)**2 + 100*(x[1]-x[0]**2)**2;

In [ ]:
# Initialize figure 
fig = plt.figure(figsize=(12, 7))
ax = fig.subplots(1,1)

# Evaluate function
X = np.linspace(-2, 2, 100)
Y = np.linspace(-1, 3, 100)
X, Y = np.meshgrid(X, Y)
Z = frb([X,Y])

# Plot the surface
surf = ax.contourf(X,Y,np.log10(Z),cmap=plt.cm.turbo,levels=100)
ax.contour(X,Y,np.log10(Z),colors='k',levels=np.arange(0,2+1e-5,0.5))
# ax.set_zlim(0, 200)
fig.colorbar(surf, shrink=0.5, aspect=10,label='log10(Z)')

In [ ]:
frb([1,1])

In [ ]:
# simple minimization
res = sp.optimize.minimize(frb,[0,0])
res

In [ ]:
res = sp.optimize.dual_annealing(frb,bounds=[[-5,5],[-5,5]])
res

In [ ]:
res = sp.optimize.shgo(frb,bounds=[[-5,5],[-5,5]])
res

### Ackley Function

In [ ]:
fack = lambda x: -20*np.exp(-0.2*np.sqrt(0.5*((x[0])**2+(x[1])**2)))-\
                np.exp(0.5*(np.cos(2*np.pi*(x[0]))+np.cos(2*np.pi*(x[1]))))+np.e+20

In [ ]:
fack([0,0]),fack([3,3]),fack([0,1])

In [ ]:
# Initialize figure 
fig = plt.figure(figsize=(12, 7))
ax = fig.subplots(1,1)

# Evaluate function
X = np.linspace(-5, 5, 100)
Y = np.linspace(-5, 5, 100)
X, Y = np.meshgrid(X, Y)
Z = fack([X,Y])

# Plot the surface
surf = ax.contourf(X,Y,Z,cmap=plt.cm.turbo,levels=100)
ax.contour(X,Y,np.log10(Z),colors='k',levels=np.arange(0,2+1e-5,0.5))
# ax.set_zlim(0, 200)
fig.colorbar(surf, shrink=0.5, aspect=10)

In [ ]:
res = sp.optimize.minimize(fack,[3,3])
res

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.subplots(1,1)
# Plot the surface
surf = ax.contourf(X,Y,Z,cmap=plt.cm.turbo,levels=100)
ax.contour(X,Y,np.log10(Z),colors='k',levels=np.arange(0,2+1e-5,0.5))
ax.plot(res.x[0],res.x[1],'s',color='white',mec='grey',markersize=10)
# ax.set_zlim(0, 200)
fig.colorbar(surf, shrink=0.5, aspect=10)

Well that's not right. What happens if we try again, with randomly seeded first guesses?

In [ ]:
ndraw = 100
resx = np.array([sp.optimize.minimize(fack,rr).x for rr in np.random.uniform(-5,5,size=(ndraw,2))])

In [ ]:
fig = plt.figure(figsize=(12, 7))
ax = fig.subplots(1,1)
# Plot the surface
surf = ax.contourf(X,Y,Z,cmap=plt.cm.turbo,levels=100)
ax.contour(X,Y,np.log10(Z),colors='k',levels=np.arange(0,2+1e-5,0.5))
ax.plot(resx[:,0],resx[:,1],'s',color='white',mec='grey',markersize=10)
# ax.set_zlim(0, 200)
fig.colorbar(surf, shrink=0.5, aspect=10)

Still not looking good. Bigger numbers go brrrr?

In [ ]:
ndraw = 2500
resx = np.array([sp.optimize.minimize(fack,rr).x for rr in np.random.uniform(-5,5,size=(ndraw,2))])

fig = plt.figure(figsize=(12, 7))
ax = fig.subplots(1,1)
# Plot the surface
surf = ax.contourf(X,Y,Z,cmap=plt.cm.turbo,levels=100)
ax.contour(X,Y,np.log10(Z),colors='k',levels=np.arange(0,2+1e-5,0.5))
ax.plot(resx[:,0],resx[:,1],'o',color='white',mec='grey',markersize=5)
# ax.set_zlim(0, 200)
fig.colorbar(surf, shrink=0.5, aspect=10)

Well now I'm more lost than ever. Still, let's check the distribution.

In [ ]:
# Start with a square Figure.
fig = plt.figure(figsize=(6, 6))
# Add a gridspec with two rows and two columns and a ratio of 1 to 4 between
# the size of the marginal axes and the main axes in both directions.
# Also adjust the subplot parameters for a square plot.
gs = fig.add_gridspec(2, 2,  width_ratios=(4, 1), height_ratios=(1, 4),
                      left=0.1, right=0.9, bottom=0.1, top=0.9,
                      wspace=0.05, hspace=0.05)
# Create the Axes.
ax = fig.add_subplot(gs[1, 0])
ax_histx = fig.add_subplot(gs[0, 0], sharex=ax)
ax_histy = fig.add_subplot(gs[1, 1], sharey=ax)

    # no labels
ax_histx.tick_params(axis="x", labelbottom=False)
ax_histy.tick_params(axis="y", labelleft=False)

# the scatter plot:
ha = ax.hist2d(x=resx[:,0],y=resx[:,1],cmap=plt.cm.BuPu,bins=np.arange(-5.5,5.6,1));

ax_histx.hist(resx[:,0], bins=np.arange(-5.5,5.6,1),edgecolor='k');
ax_histy.hist(resx[:,1], bins=np.arange(-5.5,5.6,1), orientation='horizontal',edgecolor='k');

Okay so it works with a Monte Carlo, but there's got to be a smarter way

## Annealing

Xiang, Y., Sun, D.Y., Fan, W. and Gong, X.G., 1997. Generalized simulated annealing algorithm and its application to the Thomson model. Physics Letters A, 233(3), pp.216-220.

https://www.sciencedirect.com/science/article/pii/S037596019700474X

In [ ]:
res = sp.optimize.dual_annealing(fack,bounds=[[-5,5],[-5,5]])
res

# SHGO
Endres, SC, Sandrock, C, Focke, WW (2018) “A simplicial homology algorithm for lipschitz optimisation”, Journal of Global Optimization.
https://link.springer.com/article/10.1007/s10898-018-0645-y

In [ ]:
res = sp.optimize.shgo(fack,bounds=[[-5,5],[-5,5]])
res